# Lab Notebook
## Hanqing Liu

### Background
- Input data, fastq files from pair-end HiC, two conditions:
    - proB cell
    - proB cell (treated with Gleevec)

### 02/20/2018
1. BAM files from Arthur
Aligned by STAR

2. make Tag dir for both hic exp by Arthur
~~~~bash
makeTagDirectory SOME_DIR -tbp 1 -removeSelfLigation -restrictionSite GATC -both -genome mm10 -removePEbg -removeRestrictionEnds hic_read1.bam,hic_read2.bam
~~~~

### 02/21/2018
1. Generate HiC background
~~~~bash
analyzeHiC ~/data/TAG_DIR/ -res 25000 -bgonly -cpu 2
~~~~

2. Generate interaction matrix
    - Generate whole genome matrix using low resolution (1M, 500K)
    - Generate per chrom matrix using high resolution (25K)
        - the relationship about reads length and resolution: https://www.biostars.org/p/173026/
        - Here I assume 100M reads can reach ~25K resolution
        - Generate 25K whole genome matrix would be too large (10^10). 
        - Interchrom interact are more sparse, should use lower resolution anyway.


## 02/22/2018
1. Generate interaction matrix
~~~~bash
python /home/hliu/src/hic/generate_matrix.py whole_genome --i=/home/hliu/data/proB-C-tag --o=/home/hliu/data/proB-C-matrix --name=c --cpu=12 --resolution=1000000 --bychr False -corr
python /home/hliu/src/hic/generate_matrix.py whole_genome --i=/home/hliu/data/proB-G-tag --o=/home/hliu/data/proB-G-matrix --name=g --cpu=12 --resolution=1000000 --bychr False -corr
python /home/hliu/src/hic/generate_matrix.py whole_genome --i=/home/hliu/data/proB-C-tag --o=/home/hliu/data/proB-C-matrix --name=c --cpu=1 --resolution=25000 --bychr True -corr
python /home/hliu/src/hic/generate_matrix.py whole_genome --i=/home/hliu/data/proB-G-tag --o=/home/hliu/data/proB-G-matrix --name=g --cpu=1 --resolution=25000 --bychr True -corr
~~~~

2. Generate chrom level PCA, find compartments
~~~~bash
runHiCpca.pl g.PCA /home/hliu/data/proB-G-tag/ -res 25000 -cpu 5 -genome mm10
runHiCpca.pl c.PCA /home/hliu/data/proB-C-tag/ -res 25000 -cpu 5 -genome mm10
runHiCpca.pl g.PCA.1000000 /home/hliu/data/proB-G-tag/ -res 1000000 -cpu 5 -genome mm10
runHiCpca.pl c.PCA.1000000 /home/hliu/data/proB-C-tag/ -res 1000000 -cpu 5 -genome mm10
findHiCCompartments.pl c.PCA.PC1.txt > c.PCA.PC1.compartments.txt
findHiCCompartments.pl g.PCA.PC1.txt > g.PCA.PC1.compartments.txt
~~~~

3. Generate correlation difference matrix, find diff compartments
~~~~bash
getHiCcorrDiff.pl c_vs_g /home/hliu/data/proB-C-tag/ /home/hliu/data/proB-G-tag/ -res 25000 -cpu 4
findHiCCompartments.pl /home/hliu/data/proB-C-matrix/c.PCA.PC1.txt -bg /home/hliu/data/proB-G-matrix/g.PCA.PC1.txt -corr /home/hliu/data/compare/c_vs_g.corrDiff.txt > /home/hliu/data/compare/c_vs_g_diff_compartments.txt
~~~~

4. Annotate PCA regions
~~~~bash
annotatePeaks.pl tss mm10 -gtf /home/hliu/ref/gencode.vM16.annotation.gtf -bedGraph /home/hliu/data/proB-C-matrix/c.PCA.PC1.bedGraph /home/hliu/data/proB-G-matrix/g.PCA.PC1.bedGraph /home/hliu/data/compare/c_vs_g_diff_compartments.txt > /home/hliu/data/compare/region.anno.25000.txt
~~~~

5. Analyze these result
    - see check_hic.ipynb

## 02/27/2018
1. Find significant interactions
~~~~bash
# for 25K, search intra only
findHiCInteractionsByChr.pl ~/data/proB-G-tag -res 25000 -superRes 50000 -ped ~/data/proB-C-tag -cpu 4 > significantInteractions.25000.intra.txt
# for 500K, search intra and inter
analyzeHiC ~/data/proB-G-tag -res 500000 -interactions significantInteractions.500000.txt -nomatrix -ped ~/data/proB-C-tag -cpu 8
~~~~

### Plans
- Generate plot for QC stat from Homer
- Check HiCPro
- Check higlass